In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import actr
import pickle
import random
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from itertools import groupby
import json
from IPython.display import Image 
import time

ACT-R connection has been started.


In [3]:
### Experiment part ###
def __init__(iteration):
    subject = ''

    current_list = ''

    #recalled_words = defaultdict(list)

    

    list_amount = 2   # No of lists (100,200, 1000, 2000 AND 5000)

    list_length = 5  # No of words in a list

    rehearsal_time = 1 # No of seconds for which rehearsal happens and each word is shown

    delay_words = 0.5  #delay between rehearsal and recall
    
    #delay_recall = 1

    #recall_time = 10
    
    list_keys = list(range(list_amount))
    value = []
    #recalled_words = dict.fromkeys(list_keys,value)
    
    #rehearsed_words =  defaultdict(lambda: defaultdict(int))

    word_lists_dict = defaultdict(list)
    word_indices_dict = defaultdict(dict)
   
    # Ensure there are enough unique words to create the word lists
    word_dict = {"positive": ["positive" + str(i) for i in range(999)],
                 "negative": ["negative" + str(i) for i in range(999)],
                 "neutral": ["neutral" + str(i) for i in range(999)]}
   
      # Ensure there are enough unique words to create the word lists
#     word_dict = {"positive": ["positive" + str(i) for i in range(list_length)],
#                  "negative": ["negative" + str(i) for i in range(list_length)],
#                  "neutral": ["neutral" + str(i) for i in range(list_length)]}

    all_unique_words = set()
    
    extra_list_intrusions = 0

#     filename = f'./postman/postman_agents/delay_{delay}/words_{list_length}_lists_{list_amount}_rh_time_{rehearsal_time}_rec_time_{recall_time}_delay_{delay}_{iteration}.txt'
    
#     results = {}



#     # results['x'] = {'data': [], # will be appended later in the analytics function
#     #                      'info': "Storing range(len(word_lists_dict[0])) here"}

#     results['rehearse_frequency'] = {'data': None,# will be appended later in the analytics function

#                                      'info': "Storing list(rehearse_frequency.values()) here"}

#     results['recall_probability'] = {'data': None, # will be appended later in the analytics function
#                                      'info': "Storing list(recall_probability.values()) here"}

#     results['first_recall'] = {'data': None, # will be appended later in the analytics function
#                                'info': "Storing list(first_probability.values()) here"}

#     with open(filename, 'w') as outfile:
#         json.dump(results, outfile)

#     with open(filename) as json_file:
#         results = json.load(json_file)
#         #print(results)
    
    globals().update(locals())  ## Making everything public, worst code you can ever write!!

In [4]:
def add_words(i, list_length):
    '''
    Add the words to the word lists, ensures valence categories are balanced
    '''
    global word_lists_dict,word_indices_dict

    amnt_wanted = (list_length -2)/3   # Amount of each valence wanted, minus 2 neutrals controlling for primacy
    amt_positive, amt_negative, amt_neutral, count = 0, 0, 0, 0
    while len(word_lists_dict[i]) != list_length:
        count += 1
        #print(f"...................{count,word_lists_dict[i]}")
        if count >= 9999: # IF it takes too long to create a unique list at random, start over
            word_lists_dict[i] = []
            add_words(i, list_length)
        if len(word_lists_dict[i]) == 0: # Place two neutral words at the start to control for primacy effects
            word_to_add1 = word_dict["neutral"][random.randint(0, len(word_dict["neutral"])-1)]
            word_to_add2 = word_dict["neutral"][random.randint(0, len(word_dict["neutral"])-1)]
            if word_to_add1 not in word_lists_dict[i] and word_to_add2 not in word_lists_dict[i] and word_to_add1 != word_to_add2:
                word_lists_dict[i].append(word_to_add1)
                word_lists_dict[i].append(word_to_add2)
            else:
                continue # skip this loop iteration                   
        else: 
            random_valence = random.choice(["positive", "negative", "neutral"])
            word_to_add = word_dict[random_valence][random.randint(0, len(word_dict[random_valence])-1)]
            if word_to_add not in word_lists_dict[i] and word_lists_dict[i][-1] not in word_dict[random_valence] and \
               amt_positive <= amnt_wanted and amt_negative <= amnt_wanted and amt_neutral <= amnt_wanted:
                if random_valence == "positive" and amt_positive < amnt_wanted:
                    amt_positive += 1
                elif random_valence == "negative" and amt_negative < amnt_wanted:
                    amt_negative +=1
                elif random_valence == "neutral" and amt_neutral < amnt_wanted:
                    amt_neutral +=1
                else:
                    continue # skip this loop iteration
                word_lists_dict[i].append(word_to_add)
                
    #convert the list to dictionary 
    word_indices_dict[i] = dict(zip(list(range(list_length)),word_lists_dict[i]))
    all_unique_words.update(word_lists_dict[i])
    
def create_lists(list_amount=3, list_length=2):
    '''
    Create the wordlists used during the free recall tasks 
    '''  
    global word_lists_dict 
    print(f'Creating {list_amount} lists with {list_length} words.')
    for i in range(list_amount):
        print(f'List {i+1}/{list_amount} created!', end="\r")
        add_words(i, list_length)

    # Save the dictionary to a .pickle file, so we do not have to create the word lists everytime we run the model                    
    file = open(f"word_lists/word_lists_dict_{list_length}_{list_amount}.pickle","wb")
    pickle.dump(word_lists_dict, file)
    file.close()
    return word_lists_dict

# Check if the word lists already exist, else create new word lists
def check_and_create_lists():
    global word_lists_dict
    try:
        file = open(f"word_lists/word_lists_dict_{list_length}_{list_amount}.pickle","rb")
        #file = open(f"word_lists_dict_100_items_only.pickle","rb")
        word_lists_dict = pickle.load(file)
        for k,v in word_lists_dict.items():
            #print(k,v)
            word_indices_dict[k] = dict(zip(list(range(list_length)),v))
            all_unique_words.update(v)
        #word_indices_dict[i] = dict(zip(list(range(list_length)),word_lists_dict[i]))
        file.close()
        print("\nSuccesfully loaded the word lists!\n")
    except FileNotFoundError:
        print("\nCreating word lists!\n")
        #amount_to_create = list_amount                              
        word_lists_dict = create_lists(list_amount,list_length)

def display_word_lists():
    '''
    Display the word lists loaded/created
    '''
    for key, value in word_lists_dict.items():
        print(f'List {key}:\n {value}\n')
#     for key, value in word_indices_dict.items():
#         print(f'List {key}:\n {value}\n')

def close_exp_window():
    '''
    Close opened ACT-R window
    '''
    return actr.current_connection.evaluate_single("close-exp-window")

def prepare_for_recall(): 
    '''
    Disable rehearsing productions, and clearing buffer contents to start the recalling phase 
    '''
    disable_list = ["rehearse-first", "rehearse-second", "rehearse-third", "rehearse-fourth", 
                    "rehearse-it", "skip-first", "skip-second", "skip-third", "skip-fourth"]
    for prod in disable_list:
        actr.pdisable(prod)
    
    # Setting delay in between rehearsal and recall
    actr.run(delay_recall, False) 
    for buff in ["imaginal", "retrieval", "production"]:
        actr.clear_buffer(buff)  

def setup_dm(word_list):
    '''
    Add words to declarative memory, since it can be assumed the test subjects know the English language already
    '''
    #print("\n\n############################################# Inside setup_dm i.e. Declarative Memory")
     
    colour_conversion = {'pos': 'GREEN', 'neg': 'RED', 'neu': 'BLACK'}
    for idx, word in enumerate(word_list):
        valence = ''.join([char for char in word if not char.isdigit()])[:3]
        actr.add_dm(('item'+str(idx), 'isa', 'memory', 'word', "'"+word+"'", 'valence', colour_conversion[valence]))
#         if idx == 0:
#             print("\n Emaple of a chunk added in Declarative Memory is \n")
#             print('item'+str(idx), 'isa', 'memory', 'word', "'"+word+"'", 'valence', colour_conversion[valence],"\n")
        

def setup_experiment(human=True):
    '''
    Load the correct ACT-R model, and create a window to display the words
    '''
#     print("\n\n############################################# Inside setup_experiment")
#     print(f'\nSubject = {subject}\n')  
    #print(f"\nThe model parameters are {actr.modules_parameters()}")

    if subject == "controls":
        actr.load_act_r_model("C:/Users/brata/Desktop/fyrp_model_experiments/csm_free_recall_model_experiments.lisp")
    elif subject == "depressed":
        actr.load_act_r_model("C:/Users/brata/Desktop/fyrp_model_experiments/csm_free_recall_model_depressed.lisp")

    #print("\n\n############################################# Inside setup_experiment")
    #print(f'\nLoaded Act-r model = {loaded}\n')  



    window = actr.open_exp_window("Free Recall Experiment", width=1024, height=768, visible=human) # 15inch resolution window
    actr.install_device(window) 
    return window    

def record_words_recalled(item):
    '''
    Register which words were recalled during the experiment for a specific wordlist and strip the numbers
    '''
#     print("============ACT-R parameters while the worda are recalled ====================")

#     print("============Chunks in the model====================")
#     print(actr.buffer_chunk("retrieval"))

#     print("============Chunks in the DM====================")
#     print(actr.dm()) #### Can give the name of chunks in the parameters as well
#     print(f"in record_words_recalled, the item is {item}")
    print("================Record words recalled called=================")
    print(f"The current list number is {current_list} and the items recalled are {item}")
    valence = ''.join(char for char in item if not char.isdigit())
    item_idx = ''.join(char for char in item if char.isdigit())
    recalled_words[current_list].append((valence, item_idx))

def record_words_rehearsed(item):
    '''
    Register amount of rehearsals per word for each wordlist
    '''
    print("================Record words Rehearsed called=================")
    print(f"The current list number is {current_list} and the items rehearsed are {item}")
    rehearsed_words[current_list][item] += 1



        
def create_result_dict():
    '''
    Use a module-level function, instead of lambda function, to enable pickling it
    '''
    return defaultdict(int)

In [5]:
def do_experiment(subj="depressed", human=False, wlist_amount=20):
    '''
    Run the experiment
    '''
    check_and_create_lists()
    global subject,word_lists_dict
    subject = subj
    assert wlist_amount <= len(word_lists_dict), "Chosen too many lists, choose less or create more word lists using function: create_lists()"
    
#     print("###################################################\n")
#     print("The original word list \n")
#     print(display_word_lists())
   
    
#     print("\n###################################################\n")
#     print("Experiment started, Trying to understand the flow\n")
  
    for idx, (key, value) in enumerate(word_lists_dict.items()):
        actr.reset()
        window = setup_experiment(human)
        global current_list
        current_list = idx # keep track for which list words are recalled
        setup_dm(value)   
        
        #print("============ACT-R parameters at the beginning of the experiment or before rehearsal====================")
        
        #print("============Chunks in the model====================")
        #print(actr.buffer_chunk())
        
        #print("============Chunks in the DM====================")
        #print(actr.dm()) #### Can give the name of chunks in the parameters as well
        
        print(f"\n\n===================Starting experiment for list {idx}=====================================================")
        
#         actr.add_command("retrieved-word", record_words_recalled,"Retrieves recalled words.")
#         actr.add_command("rehearsed-word", record_words_rehearsed,"Retrieves rehearsed words.")
#         print("\n##################  Model started rehearsal ")

        for word in value:
            if "neutral" in word:
                color = "black"
            elif "positive" in word:
                color = "green"
            else:
                color = "red"
            print(f"\n\n Word displayed at the Window {word}")
            actr.add_text_to_exp_window(window, word, x=475-len(word) , y=374, color=color, font_size=20) # change later 
            actr.run(rehearsal_time, human) # True when choosing Human, False when choosing differently
#             print("====================During Word Rehearsals Buffers Goal and Imaginal==================")
#             print(actr.buffer_chunk("goal","imaginal"))
#             print("====================During Word Rehearsals Why Not==================")
#             print(actr.whynot("rehearse-first","rehearse-second","rehearse-third","rehearse-fourth"))
            actr.run(delay_words, human)  #
            actr.clear_exp_window(window)
            
#         #actr.run(delay, human)  # 500-ms blank screen  
#         print("\n\n==============Starting Recall================")
#         prepare_for_recall() #Disable the retrieval productions and clear the buffers, takes 1 sec in doing so.   
#         actr.remove_command("rehearsed-word")
#         actr.goal_focus("startrecall") # set goal to start recalling
#         actr.run(recall_time, human)  
#         actr.remove_command("retrieved-word")
        print(f"============Chunks in the DM after  {idx+1}/{wlist_amount}====================")
        print(actr.dm()) #### Can give the name of chunks in the parameters as well
        print(f'\n\nExperiment {idx+1}/{wlist_amount} completed!')
        
#         print("============ACT-R parameters after rehearsal or before recall ====================")
        
#         print("============Chunks in the model====================")
#         print(actr.buffer_chunk('goal'))
        
        #print("============Chunks in the DM====================")
        #print(actr.dm()) #### Can give the name of chunks in the parameters as well
#         print("\n##################  Model finished rehearsal, list of rehearsed words is ")
#         print(f'{rehearsed_words}\n')
#         print("\n##################  Model started recall ")
        
#         print("============ACT-R parameters after recall ====================")
#         print(actr.whynot('start-recall'))
#         print("============Chunks in the model====================")
#         print(actr.buffer_chunk('goal'))
        
#         print("============Chunks in the DM====================")
#         print(actr.dm()) #### Can give the name of chunks in the parameters as well
        
#         print("\n##################  Model finished recall, list of recalled words is ")
#         print(f'{recalled_words}\n')
        if idx == wlist_amount-1: # run for a chosen amount of word lists
            break
    close_exp_window() # close window at end of experiment     

#     avg_recalled, avg_recalled_unique = 0, 0
#     for key, val in recalled_words.items():
#         avg_recalled += len(val)
#         avg_recalled_unique += len(set(val))
#     #    print(f'\nList {key} (length={len(val)}, unique={len(set(val))})')
#     print(f'Avg. Amount of words recalled = {avg_recalled//wlist_amount}')
#     print(f'Avg. Amount of unique words recalled = {avg_recalled_unique//wlist_amount}')
    
    #print(f'recalled words {recalled_words}')
    
#     result = analysis(list_amount, False)        

#     for key, val in result.items():
#         print(f'{key} = {dict(val)}')
#     print()
 

    print("\n\n#############################################")
    print(f'\n[{subj}] Results!\n')  

In [8]:
num_agents = 1
for agent in range(num_agents):
    print("------------------------------")
    print(f"Started for agent_{agent}")
    __init__(agent)
    time.sleep(1)
    do_experiment('controls',False,list_amount)

------------------------------
Started for agent_0

Succesfully loaded the word lists!



===================Starting experiment for list 0=====================================================


 Word displayed at the Window neutral864
     0.000   GOAL                   SET-BUFFER-CHUNK GOAL GOAL NIL
     0.000   VISION                 SET-BUFFER-CHUNK VISUAL-LOCATION VISUAL-LOCATION0 NIL
     0.050   PROCEDURAL             PRODUCTION-FIRED FIND-UNATTENDED-WORD
     0.050   VISION                 SET-BUFFER-CHUNK VISUAL-LOCATION VISUAL-LOCATION0
     0.185   VISION                 SET-BUFFER-CHUNK VISUAL TEXT0
     0.235   PROCEDURAL             PRODUCTION-FIRED ATTEND-WORD
     0.320   VISION                 SET-BUFFER-CHUNK VISUAL TEXT1
     0.370   PROCEDURAL             PRODUCTION-FIRED HIGH-SECOND
     0.570   IMAGINAL               SET-BUFFER-CHUNK IMAGINAL CHUNK0
     0.620   PROCEDURAL             PRODUCTION-FIRED ADD-TO-MEMORY-2
     0.620   ------                 Stopped bec